In [ ]:
install.packages("data.table")
install.packages("realxl")
install.packages("dplyr")
## Importing packages
library(data.table) # Import big files (CSV or Text) 
library(readxl) # Import Excel files
library(dplyr)

In [ ]:
                        #creating a file and its variable 

house <- read.csv("../input/kaggle-projectindore/IndoreHP_Train.csv")
test <- read.csv("IndoreHP_Test.csv")

In [ ]:
# drop zn
#------- crim-------------( ok )

log_crim<- log(house$CRIM)
log_crim
boxplot(log_crim)
length(log_crim) # 350

In [ ]:
summary(house$INDUS)

In [ ]:
#---------- indus ( ok )
bench_indus<- 10.2636 +1.5 * IQR(house$INDUS)
bench_indus
t_indus <- ifelse(house$INDUS > bench_indus,bench_indus,house$INDUS)
t_indus
boxplot(t_indus)
summary(t_indus)
length(t_indus) # 350

In [ ]:
#------------ no2 ( oka )
bench_NO2<- 0.5429 +1.5 * IQR(house$nitric.oxides.concentration)
bench_NO2
t_NO2 <- ifelse(house$nitric.oxides.concentration > bench_NO2,bench_NO2,house$nitric.oxides.concentration)
t_NO2
boxplot(t_NO2)
length(t_NO2) #350

In [ ]:

#------------ dwelling room  ( okay )
bench_roomd<- 6.747 +1.5 * IQR(house$X.rooms.dwelling)
bench_roomd
t_roomd <- house %>% select (X.rooms.dwelling) %>% filter ( X.rooms.dwelling <= bench_roomd)
boxplot(t_roomd)
t_roomd <- ifelse(house$X.rooms.dwelling > bench_roomd,bench_roomd,house$X.rooms.dwelling)
t_roomd
boxplot(t_roomd)
length(t_roomd) #350

In [ ]:
attach(house)
#-------------- age ( oka )
boxplot( house$AGE)
length(house$AGE) #350

#---------------- dis ( oka )
boxplot(house$DIS)
length(house$DIS) # 350
#---------------tax ( oka)
summary(house$TAX)
boxplot(house$TAX)
length(house$TAX)  #350

#--------------PTRATIO
summary(PTRATIO)
boxplot(PTRATIO)
length( PTRATIO) #350

In [ ]:
#------------- rad (  )

boxplot(house$RAD)
bench_rad<- 5 +1.5 * IQR(house$RAD)
bench_rad
bench_radL <- 4 - 1.5 * IQR(house$RAD)
bench_radL 
summary(house$RAD)
change_rad <- function(x)
{
  if (x >bench_rad){return (bench_rad)}
  else if (x < bench_radL ){ return (bench_radL)}
  else {return(x)}
}
t_rad <- (sapply(house$RAD , change_rad))
boxplot(t_rad)
hist(t_rad)
length(t_rad) # 350

In [ ]:
#------------------ B  ( work on this )

bench_B<- 375.53 - 1.5 * IQR(house$B)
bench_B
house$B
change_B <- function(x)
{
  if (x < bench_B){return (bench_B)}
  else {return(x)}
}
t_B <- (sapply(house$B , change_B))
boxplot(t_B)

length(t_B) #350

In [ ]:
#---------------- lstat ( okay )
boxplot(LSTAT)
bench_lstat <- 13.245 + IQR(LSTAT)
bench_lstat
house %>% count(LSTAT < bench_lstat )
t_stat <- house %>% select(LSTAT) %>% filter(LSTAT <= bench_lstat)
t_lstat <- log(house$LSTAT)
boxplot(t_lstat)
summary(LSTAT)
length ( t_lstat) # 350

In [ ]:
#------------ medv

bench_MEDV<-  28.52 + 1.5 * IQR(house$MEDV)
bench_MEDV
t_MEDV <- ifelse(house$MEDV > bench_MEDV,bench_MEDV,house$MEDV)
boxplot(t_MEDV)
 length( t_MEDV) #350

In [ ]:
#--------------- combining columns into data frame -----------------------------------

  data.house <- data.frame(log_crim, t_indus, 
    t_NO2, t_roomd, house$AGE, house$DIS, house$TAX, t_rad,
    house$PTRATIO, t_B, t_lstat , t_MEDV,house $RIVER_FLG)
colnames(data.house) 
col_name <- c("CRIM", "INDUS","NO2","ROOM_DIM","AGE","DIS","TAX","RAD","PT_RATIO","B","LSTAT","MEDV","RIVER_FLG")
names(data.house) <- col_name
colnames(data.house)
data.house$RIVER_FLG <- as.factor(data.house$RIVER_FLG)
str(data.house)

In [ ]:
#-----------------pre- regresssion  confirmation---------------------------------------------


model_house <- lm(MEDV~.,data = data.house)
summary(model_house)

model_house2 <- lm ( MEDV~ CRIM + INDUS + NO2+ROOM_DIM+AGE+DIS+TAX+PT_RATIO+LSTAT,data = data.house)
summary(model_house2)
plot(model_house2)

model_house3 <- lm ( MEDV~ CRIM + INDUS + NO2+ROOM_DIM+DIS+TAX+PT_RATIO+LSTAT,data = data.house)
summary(model_house3)

model_house4<- lm ( MEDV~ CRIM + INDUS + NO2+ROOM_DIM+DIS+PT_RATIO+LSTAT,data = data.house)
summary(model_house4)


null = lm(MEDV~1,data = data.house)
full = model_house

aicmodel = step(null,scope=list(lower=null,upper=full),
                direction="both",k=2)
aicmodel2 = step(null,scope=list(lower=null,upper=full),
                direction="backward",k=2)
summary(aicmodel)

model_house5 <- lm (MEDV ~ ROOM_DIM + LSTAT + PT_RATIO + TAX + DIS + AGE + INDUS + 
                      CRIM + NO2, data = data.house)
summary(model_house5)

test[1,]
col_name2 <- c("ID","CRIM", "ZN","INDUS","RIVER_FLG","NO2","ROOM_DIM","AGE","DIS","TAX","PT_RATIO","B","LSTAT")
names(test) <- col_name2

In [ ]:
confint(model_house5,level = .95)

In [ ]:
ci=predict(model_house5,data.house, interval = "confidence",level = .99)
head(ci)


In [ ]:
data.house %>% mutate(predicted_MEDV=predict(model_house5,data.house),X1=ci[,1],lwr=ci[,2],upr=ci[,3]) -> New_train
New_train

In [ ]:
New_train %>% mutate(flag = ifelse(MEDV >= lwr & MEDV <=upr,1 , 0)) -> New_train

sum(New_train$flag)

**testing for test data**


In [ ]:
test<-read.csv("../input/kaggle-projectindore/IndoreHP_Test.csv")
test


In [ ]:
colnames(data.house)
colnames(test)
names(test) <- c('ID','CRIM','ZN','INDUS','RIVER_FLG','NO2','ROOM_DIM','AGE','DIS','RAD','TAX','PT_RATIO','B','LSTAT')
test

In [ ]:

test %>% mutate(CRIM = log(CRIM))



In [ ]:
summary(test$INDUS)
bencht_indus<- 18.643  +1.5 * IQR(house$INDUS)
 bencht_indus
 T_INDUS <- ifelse(test$INDUS > bencht_indus,bencht_indus,test$INDUS)
 boxplot(T_INDUS)


In [ ]:
summary(test$NO2)
benchT_NO2<- 0.7104 +1.5 * IQR(house$nitric.oxides.concentration)
benchT_NO2
T_NO2 <- ifelse(house$nitric.oxides.concentration > bench_NO2,bench_NO2,house$nitric.oxides.concentration)
T_NO2
boxplot(T_NO2)

In [ ]:
summary(test$ROOM_DIM)
boxplot(test$ROOM_DIM)
benchT_roomd<- 6.585 +1.5 * IQR(test$NO2)
benchT_roomd
T_roomd <- ifelse(test$NO2 > benchT_roomd,benchT_roomd,test$NO2)
boxplot(T_roomd)
#length(T_roomd) 350

In [ ]:
colnames(test)

In [ ]:
NEW_TEST <- test %>% mutate(CRIM = log(CRIM),INDUS=ifelse(test$INDUS > bench_indus,bench_indus,test$INDUS),RIVER_FLG = as.factor(RIVER_FLG),NO2 =ifelse(test$NO2 > bench_NO2,bench_NO2,test$NO2),ROOM_DIM =ifelse(test$ROOM_DIM > bench_roomd,bench_roomd,test$ROOM_DIM),RAD = (sapply(test$RAD , change_rad)) ,B=(sapply(test$B , change_B)) ,LSTAT=log(LSTAT))
NEW_TEST

In [ ]:
ci=predict(model_house5,test, interval = "confidence",level = .95)

In [ ]:
test %>% mutate(MEDV=predict(model_house5,test)) -> SHUBHAM_KOSHAL_FILE

In [ ]:
ncol(SHUBHAM_KOSHAL_FILE)
latest_file <- SHUBHAM_KOSHAL_FILE %>% mutate(MEDV = abs(MEDV))  

In [ ]:
latest_file

In [ ]:
write.csv(latest_file,"shubham_koshal_new.csv",row.names = FALSE)